## 1 원-핫 인코딩(One-Hot Encoding)

### 1.1 직접 구현해보기

"원숭이, 바나나, 사과"로 원-핫 인코딩을 한다면

In [ ]:
# 인코딩 대상 단어들을 담은 리스트
word_ls = ['원숭이', '바나나', '사과', '개', '고양이']

In [2]:
from collections import defaultdict
import numpy as np

def on_hot_encode(word_ls):
    
    # 고유 단어와 인덱스를 매칭시켜주는 사전 생성
    word2id_dic = defaultdict(lambda:len(word2id_dic))
    
    # {단어 : 인덱스} 사전 구축
    for word in word_ls:
        word2id_dic[word]
        
    n_uniue_words = len(word2id_dic)  # 고유한 단어의 갯수
    one_hot_vectors = np.zeros((len(word_ls), n_uniue_words))  # 원핫-벡터를 만들기 위해 비어있는 벡터 생성
    
    for i, word in enumerate(word_ls):
        index = word2id_dic[word]  # 해당 단어의 고유 인덱스
        one_hot_vectors[i, index] = 1  # 해당 단어의 고유 인덱스에만 1을 더해줌
        
    return one_hot_vectors

In [3]:
one_hot_vectors = on_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

### 1.2 "코끼리"라는 단어가 추가된다면?

In [4]:
word_ls.append('코끼리')

In [5]:
one_hot_vectors = on_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

### 1.3 sklearn 활용

In [6]:
# sklearn을 활용한 one-hot encoding
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# 예제 데이터 배열
values = array(word_ls)
print(values)

# 문자열에 숫자를 붙임
label_enc = LabelEncoder()
int_enc = label_enc.fit_transform(values)
print(int_enc)

# binary encode
onehot_enc = OneHotEncoder(sparse=False)
int_enc = int_enc.reshape(len(int_enc), 1)  # n:1 matrix로 변환
print(int_enc)
onehot_enc = onehot_enc.fit_transform(int_enc)
print(onehot_enc)

# one-hot encoding의 첫번째 배열을 값을 역으로 산출
inverted = label_enc.inverse_transform([argmax(onehot_enc[0, :])])
print(inverted)

['원숭이' '바나나' '사과' '개' '고양이' '코끼리']
[4 2 3 0 1 5]
[[4]
 [2]
 [3]
 [0]
 [1]
 [5]]
[[0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]
['원숭이']


## 2 밀집 벡터 (Denses Vector)

### 2.1 유사도 계산

#### 2.1.1 유클리디언 거리(Euclidean distance)
두 벡터사이의 직선 거리. 피타고라스 정리를 생각하면 이해하기 쉬움

In [7]:
word_embedding_dic = {
    '사과' : [1.0, 0.5],
    '바나나' : [0.9, 1.2],
    '원숭이' : [0.5, 1.5]
}

In [8]:
import numpy as np
def euclidean_dist(x, y):
    x = np.array(x)
    y = np.array(y)
    return np.sqrt(np.sum((x-y)**2))

# 사과와 바나나의 코사인 유사도
euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나'])

0.7071067811865475

#### 2.1.2 자카드 유사도(Jaccard index)

In [9]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'

# 토큰화를 수행합니다.
token_s1 = s1.split()
token_s2 = s2.split()

union = set(token_s1).union(set(token_s2))
print(union)

intersection = set(token_s1).intersection(set(token_s2))
print(intersection)

print(len(intersection)/len(union)) # 2를 6로 나눔.

{'원숭이는', '싫어합니다.', '대부분', '코주부', '바나나를', '좋아합니다.'}
{'원숭이는', '바나나를'}
0.3333333333333333


#### 2.1.3 코사인 유사도(Consine Similarity)
- 두 벡터간의 유사도를 측정하는 방법 중 하나
- 두 벡터 사이의 코사인을 측정
- 0도 = 1,  90도 = 0,  180도 = -1  ==> 1에 가까울수록 유사도가 높음(각도가 작을수록)

In [12]:
def cosine_similarity(x, y):
    # x와 y, 두 벡터의 코사인 유사도를 계산하는 함수
    nominator = np.dot(x, y)  # 분자
    denominator = np.linalg.norm(x)*np.linalg.norm(y)
    return nominator/denominator

In [13]:
a = np.array([1, 2])
b = np.array([3, 4])
np.dot(a, b)

11

In [14]:
a = np.array([1, 2])
np.linalg.norm(a)

2.23606797749979

In [29]:
# 사과와 바나나의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나']))

0.8944271909999159
0.7071067811865475


## 3 TF-IDF를 활용한 단어 벡터

### 3.1 직접 구현하기

In [1]:
d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"

In [5]:
import numpy as np

# document 내 토큰이 등장한 빈도수 계산
def f(t, d):
    return d.count(t)

# tf 계산
def tf(t, d):
    return f(t, d)/len(d)

# idf 계산
def idf(t, D):
    numerator = len(D)
    denominator = len([True for d in D if t in d])
    return np.log(numerator/denominator)

# tf-idf 계산
def tfidf(t, d, D):
    return tf(t, d) * idf(t, D)

# 공백을 기준으로 토큰화
def tokenizer(d):
    return d.split()

# tfidf 계산
def tfidfScorer(D):
    docs = [tokenizer(d) for d in D]
    print("docs : ", docs)
    result = []
    for d in docs: 
        result.append([(t, tfidf(t, d, docs)) for t in d])
    return result

corpus = [d1, d2]

for i, doc in enumerate(tfidfScorer(corpus)):
    print("================== document [%d] =======================" % i)
    print(doc)

docs :  [['The', 'cat', 'sat', 'on', 'my', 'face', 'I', 'hate', 'a', 'cat'], ['The', 'dog', 'sat', 'on', 'my', 'bed', 'I', 'love', 'a', 'dog']]
================== document [0] =======================
[('The', 0.0), ('cat', 0.13862943611198905), ('sat', 0.0), ('on', 0.0), ('my', 0.0), ('face', 0.06931471805599453), ('I', 0.0), ('hate', 0.06931471805599453), ('a', 0.0), ('cat', 0.13862943611198905)]
================== document [1] =======================
[('The', 0.0), ('dog', 0.13862943611198905), ('sat', 0.0), ('on', 0.0), ('my', 0.0), ('bed', 0.06931471805599453), ('I', 0.0), ('love', 0.06931471805599453), ('a', 0.0), ('dog', 0.13862943611198905)]


### 3.2 sklearn 활용

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
count_vect = CountVectorizer()
countv = count_vect.fit_transform(corpus)
print(countv.toarray())  # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(count_vect.vocabulary_)  # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다.

[[0 2 0 1 1 0 1 1 1 1]
 [1 0 2 0 0 1 1 1 1 1]]
{'the': 9, 'cat': 1, 'sat': 8, 'on': 7, 'my': 6, 'face': 3, 'hate': 4, 'dog': 2, 'bed': 0, 'love': 5}


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]
tfidf_vect = TfidfVectorizer().fit(corpus)
tfidfv = tfidf_vect.transform(corpus)
print(tfidfv.toarray())
print(tfidf_vect.vocabulary_)

[[0.         0.70600557 0.         0.35300279 0.35300279 0.
  0.25116439 0.25116439 0.25116439 0.25116439]
 [0.35300279 0.         0.70600557 0.         0.         0.35300279
  0.25116439 0.25116439 0.25116439 0.25116439]]
{'the': 9, 'cat': 1, 'sat': 8, 'on': 7, 'my': 6, 'face': 3, 'hate': 4, 'dog': 2, 'bed': 0, 'love': 5}


### 3.3 gensim 활용

In [20]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim import corpora

d1 = "The cat sat on my face I hate a cat"
d2 = "The dog sat on my bed I love a dog"
corpus = [d1, d2]

doc_ls = [doc.split() for doc in corpus]
id2word = corpora.Dictionary(doc_ls)  # fit dictionary
corpus = [id2word.doc2bow(doc) for doc in doc_ls]  # convert corpus to BoW format

tfidf = TfidfModel(corpus)  # fit model
vector = tfidf[corpus[0]]  # apply model to the first corpus document

In [21]:
print(tfidf)
print("==============")
print(vector)

TfidfModel(num_docs=2, num_nnz=18)
[(3, 0.8164965809277261), (4, 0.4082482904638631), (5, 0.4082482904638631)]
